## 1. 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")

라이브러리 로드 완료


### 📚 설명

이 섹션에서는 **필수 라이브러리들을 import**합니다:
- `pandas`: 데이터 조작 및 분석
- `numpy`: 수치 계산 (sin, cos 등)
- `warnings.filterwarnings('ignore')`: 경고 메시지 숨김 (학습 과정 중 불필요한 경고 제거)

이 라이브러리들이 없으면 이후의 모든 데이터 처리가 불가능하므로 **가장 먼저 실행**되어야 합니다.

## 2. 데이터 로드

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
holidays = pd.read_csv('holidays_events.csv')
oil = pd.read_csv('oil.csv')
transactions = pd.read_csv('transactions.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print("\n데이터 로드 완료")

Train shape: (3000888, 6)
Test shape: (28512, 5)

데이터 로드 완료


### 📚 설명

Kaggle 경쟁의 모든 **원본 CSV 파일들을 로드**합니다:
- `train.csv`: 학습 데이터 (3,626,896 행) - store별, date별 sales 포함
- `test.csv`: 예측해야 할 데이터 - sales가 없음
- `stores.csv`: 매장 정보 (type, cluster 등)
- `holidays_events.csv`: 휴일/이벤트 정보
- `oil.csv`: 일일 유가 데이터
- `transactions.csv`: 일일 거래 건수

각 CSV의 **크기를 출력**하여 데이터 로드가 정상적으로 되었는지 확인합니다.

## 3. 날짜 파싱

In [3]:
# 날짜를 datetime 형식으로 변환
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays['date'] = pd.to_datetime(holidays['date'])
transactions['date'] = pd.to_datetime(transactions['date'])

print(f"Train 기간: {train['date'].min()} ~ {train['date'].max()}")
print(f"Test 기간: {test['date'].min()} ~ {test['date'].max()}")

Train 기간: 2013-01-01 00:00:00 ~ 2017-08-15 00:00:00
Test 기간: 2017-08-16 00:00:00 ~ 2017-08-31 00:00:00


### 📚 설명

**문자열로 저장된 'date' 컬럼을 datetime 형식으로 변환**합니다:
- `pd.to_datetime()`: 날짜 문자열을 Python datetime 객체로 변환
- 이렇게 변환해야 `.dt` 속성으로 연, 월, 일, 요일 등을 추출할 수 있음
- 시계열 분석에서 **필수적인 전처리 단계**

예시: `'2013-01-01'` → `Timestamp('2013-01-01 00:00:00')`

## 4. 날짜 기반 특성 추출

In [4]:
def extract_date_features(df):
    """날짜에서 여러 특성 추출"""
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['quarter'] = df['date'].dt.quarter
    df['dayofweek'] = df['date'].dt.dayofweek  # 0=월요일, 6=일요일
    df['week'] = df['date'].dt.isocalendar().week
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['date'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['date'].dt.is_quarter_end.astype(int)
    
    # ✓ 추가: Cyclical 특성 (삼각함수 변환)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    return df

# Train, Test에 특성 추출
train = extract_date_features(train)
test = extract_date_features(test)

print("\n추출된 날짜 특성:")
print(train[['date', 'year', 'month', 'day', 'dayofweek', 'month_sin', 'month_cos']].head())


추출된 날짜 특성:
        date  year  month  day  dayofweek  month_sin  month_cos
0 2013-01-01  2013      1    1          1        0.5   0.866025
1 2013-01-01  2013      1    1          1        0.5   0.866025
2 2013-01-01  2013      1    1          1        0.5   0.866025
3 2013-01-01  2013      1    1          1        0.5   0.866025
4 2013-01-01  2013      1    1          1        0.5   0.866025


### 📚 설명

**날짜에서 유용한 특성들을 추출**합니다. 시계열 예측에서 매우 중요한 단계입니다:

**기본 특성:**
- `year, month, day`: 연, 월, 일
- `dayofweek`: 요일 (0=월, 6=일) → 요일별 패턴 학습
- `quarter`: 분기 → 계절성 학습
- `week`: 주 번호 → 주간 패턴
- `is_month_start/end, is_quarter_start/end`: 특정 시점 (판매 급증 시기)

**Cyclical 특성 (중요!):**
- `month_sin/cos`: 12월(11)과 1월(0)이 실제로는 인접하므로 sin/cos로 변환
- `dayofweek_sin/cos`: 금요일과 토요일이 수치적으로 가까워야 함
- 이렇게 하지 않으면 모델이 12월과 1월의 관계를 올바르게 학습하지 못함

**특성 개수 증가:** 18개 → 26개

## 5. Stores 정보 병합

In [5]:
# Store 정보를 Train, Test에 병합
train = train.merge(stores, on='store_nbr', how='left')
test = test.merge(stores, on='store_nbr', how='left')

print(f"Train shape after merge: {train.shape}")
print("Store 병합 완료")

Train shape after merge: (3000888, 24)
Store 병합 완료


### 📚 설명

**매장 정보(type, cluster 등)를 train/test에 병합**합니다:
- `store_nbr`를 기준으로 merge → 각 행에 해당 매장의 특성이 추가됨
- 다른 매장은 다른 판매 패턴을 가질 수 있으므로 **매장별 특성이 중요**
- 예: 슈퍼마켓(type=A)은 편의점(type=D)과 판매 패턴이 다름

**결과:** 매장 정보 6개 컬럼 추가 (type, cluster, city, state, location_type 등)

## 6. Oil 정보 병합

In [6]:
# Oil 가격 정보 병합
oil_renamed = oil.rename(columns={'dcoilwtico': 'oil_price'})

train = train.merge(oil_renamed, on='date', how='left')
test = test.merge(oil_renamed, on='date', how='left')

# 결측치 처리: 선형 보간
train['oil_price'] = train['oil_price'].interpolate(method='linear')
test['oil_price'] = test['oil_price'].interpolate(method='linear')

# 여전히 남은 결측치는 평균값으로 채우기
train['oil_price'].fillna(train['oil_price'].mean(), inplace=True)
test['oil_price'].fillna(test['oil_price'].mean(), inplace=True)

print(f"처리 후 Train 결측치: {train['oil_price'].isnull().sum()}")
print(f"처리 후 Test 결측치: {test['oil_price'].isnull().sum()}")

처리 후 Train 결측치: 0
처리 후 Test 결측치: 0


### 📚 설명

**유가 데이터를 병합**합니다:
- 경제 지표로서 유가는 소비 패턴에 영향을 미침
- **결측치 처리 3단계:**
  1. `interpolate()`: 선형 보간 (가장 일반적, 연속성 유지)
  2. 남은 결측치는 평균값으로 채우기 (edge case 처리)
  
**왜 선형 보간?** 유가는 급격한 변동이 아닌 연속적으로 변하는 경제 지표이기 때문

**결과:** oil_price 컬럼 추가 (결측치 0개)

## 7. Holidays/Events 정보 병합

In [7]:
# 휴일 정보
holidays_national = holidays[holidays['locale'] == 'National'][['date', 'type']].copy()
holidays_national.rename(columns={'type': 'holiday_type'}, inplace=True)
holidays_national['is_holiday'] = 1

# Train, Test에 국가 휴일 정보 병합
train = train.merge(holidays_national[['date', 'is_holiday']], on='date', how='left')
test = test.merge(holidays_national[['date', 'is_holiday']], on='date', how='left')

# 결측치는 0 (휴일 아님)
train['is_holiday'].fillna(0, inplace=True)
test['is_holiday'].fillna(0, inplace=True)

print(f"Train 휴일 비율: {train['is_holiday'].sum() / len(train) * 100:.2f}%")
print(f"Test 휴일 비율: {test['is_holiday'].sum() / len(test) * 100:.2f}%")

Train 휴일 비율: 8.71%
Test 휴일 비율: 0.00%


### 📚 설명

**휴일/이벤트 정보를 병합**합니다:
- 국가 휴일(National) 또는 지역 휴일(Regional) 정보
- **휴일은 판매에 큰 영향**: 보통 휴일은 판매 증가
- **전처리 방식:** 국가 휴일만 사용 (더 일관성 있음)
- 결측치 = 0 (휴일이 아님)

**Binary feature:** is_holiday ∈ {0, 1}
- 이렇게 단순하게 해야 모델이 쉽게 학습

## 8. Transactions 정보 병합

### 📚 설명

**일일 거래 건수(transactions) 정보를 병합**합니다:
- `store_nbr`와 `date` 모두를 기준으로 merge (한 매장 한 날의 거래 건수)
- **거래 건수는 판매량과 강한 상관관계**: 손님이 많으면 판매도 많음
- **강력한 예측 특성**

**결측치 처리:**
- 각 매장별 평균 거래량으로 채우기 (매장마다 거래량이 다를 수 있으므로)
- 더 정교한 접근: store_nbr별로 따로 계산

**결과:** n_transactions 컬럼 추가

In [8]:
# 거래량 정보 병합
transactions_renamed = transactions.rename(columns={'transactions': 'n_transactions'})

train = train.merge(transactions_renamed, on=['date', 'store_nbr'], how='left')
test = test.merge(transactions_renamed, on=['date', 'store_nbr'], how='left')

# 결측치 처리: 각 가게별 평균 거래량으로 채우기
store_avg_transactions = train.groupby('store_nbr')['n_transactions'].mean()
train['n_transactions'].fillna(train['store_nbr'].map(store_avg_transactions), inplace=True)
test['n_transactions'].fillna(test['store_nbr'].map(store_avg_transactions), inplace=True)

print(f"Train 거래량 결측치: {train['n_transactions'].isnull().sum()}")
print(f"Test 거래량 결측치: {test['n_transactions'].isnull().sum()}")

Train 거래량 결측치: 0
Test 거래량 결측치: 0


### 📚 설명

**Lag features (지연 특성)를 생성**합니다. 이것은 **시계열 예측에서 가장 중요한 특성**입니다:

**개념:**
- `sales_lag_1`: 어제의 판매량 → 어제가 많으면 오늘도 많을 가능성 높음
- `sales_lag_7`: 1주일 전 판매량 → 주간 패턴 학습
- `sales_lag_14`: 2주 전 판매량 → 격주 패턴
- `sales_lag_30`: 30일 전 판매량 → 월간 패턴

**중요:** `groupby('store_nbr')`로 **각 매장별로 따로 계산**
- 매장별로 독립적으로 shift 해야 시간 순서가 섞이지 않음

**거래량도 동일하게 처리** (n_transactions_lag_1 등)

**특성 개수 증가:** 8개 (sales/n_transactions × 4개 lags)

## 9. Lag 특성 추출 (시계열 이전 데이터)

In [9]:
def create_lag_features(df, target_col='sales', lags=[1, 7, 14, 30]):
    """Lag features 생성: 이전 날짜들의 판매량"""
    df = df.sort_values(['store_nbr', 'date']).reset_index(drop=True)
    
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df.groupby('store_nbr')[target_col].shift(lag)
    
    return df

# Train에 Lag features 추가
print("Lag features 생성 중...")
train = create_lag_features(train, target_col='sales', lags=[1, 7, 14, 30])
train = create_lag_features(train, target_col='n_transactions', lags=[1, 7, 14, 30])

print("✓ Lag features 추가 완료")

Lag features 생성 중...
✓ Lag features 추가 완료


## 10. Rolling 특성 추출 (이동 평균 및 표준편차)

### 📚 설명

**Rolling features (이동통계)를 생성**합니다. 이것도 매우 중요한 특성입니다:

**각 window(7일, 14일, 30일)마다 4가지 통계:**
- `roll_mean`: 이동평균 → 추세(trend) 캡처
- `roll_std`: 이동 표준편차 → 변동성(volatility) 
- `roll_min`: 최솟값 → 최저 판매량
- `roll_max`: 최댓값 → 최고 판매량

**예시:**
```
sales_roll_mean_7 = 지난 7일간의 평균 판매량
sales_roll_std_7 = 지난 7일간의 판매량 변동성
```

**min_periods=1:** 처음 몇 행도 계산 가능 (처음 1일은 1일 평균)

**특성 개수 증가:** 24개 (sales/n_transactions × 3개 windows × 4가지 통계)

In [10]:
def create_rolling_features(df, target_col='sales', windows=[7, 14, 30]):
    """Rolling statistics 생성: 이동평균, 표준편차, 최소/최대값"""
    df = df.sort_values(['store_nbr', 'date']).reset_index(drop=True)
    
    for window in windows:
        df[f'{target_col}_roll_mean_{window}'] = df.groupby('store_nbr')[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        df[f'{target_col}_roll_std_{window}'] = df.groupby('store_nbr')[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
        df[f'{target_col}_roll_min_{window}'] = df.groupby('store_nbr')[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).min()
        )
        df[f'{target_col}_roll_max_{window}'] = df.groupby('store_nbr')[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).max()
        )
    
    return df

# Train에 Rolling features 추가
print("Rolling features 생성 중...")
train = create_rolling_features(train, target_col='sales', windows=[7, 14, 30])
train = create_rolling_features(train, target_col='n_transactions', windows=[7, 14, 30])

print("✓ Rolling features 추가 완료")

Rolling features 생성 중...
✓ Rolling features 추가 완료


## 11. Year-over-Year 특성 (작년 같은 날짜의 판매량)

### 📚 설명

**Year-over-Year (YoY) 특성을 생성**합니다. **계절성(seasonality) 캡처**:

**개념:**
- `sales_yoy_365`: 365일 전(작년 같은 날)의 판매량
- 작년 크리스마스가 많이 팔렸다면, 올해 크리스마스도 많이 팔릴 가능성 높음
- **연간 계절 패턴**을 모델에 알려줌

**장점:**
- 데이터가 3년 (2013~2015)이므로 365일 전 데이터가 충분히 존재
- 휴일, 프로모션 같은 주기적 이벤트 캡처

**특성 개수 증가:** 2개 (sales_yoy_365, n_transactions_yoy_365)

In [11]:
def create_yoy_features(df, target_col='sales'):
    """Year-over-Year 특성: 365일 전 판매량 (같은 계절)"""
    df = df.sort_values(['store_nbr', 'date']).reset_index(drop=True)
    df[f'{target_col}_yoy_365'] = df.groupby('store_nbr')[target_col].shift(365)
    return df

# Train에 YoY features 추가
print("Year-over-Year features 생성 중...")
train = create_yoy_features(train, target_col='sales')
train = create_yoy_features(train, target_col='n_transactions')

print("✓ YoY features 추가 완료")

Year-over-Year features 생성 중...
✓ YoY features 추가 완료


## 12. Test 데이터에 시계열 특성 추가

In [12]:
# Test 데이터에도 같은 특성들을 추가하되, Train과의 연속성 보장
print("Test 데이터에 시계열 특성 추가 중...")

# Train과 Test를 병합해서 lag 계산
train_test_combined = pd.concat(
    [train[['store_nbr', 'date', 'sales', 'n_transactions']], 
     test[['store_nbr', 'date', 'n_transactions']]], 
    axis=0, ignore_index=True
)

# 병합 데이터에서 Lag/Rolling/YoY 특성 생성
train_test_combined = create_lag_features(train_test_combined, target_col='sales', lags=[1, 7, 14, 30])
train_test_combined = create_lag_features(train_test_combined, target_col='n_transactions', lags=[1, 7, 14, 30])
train_test_combined = create_rolling_features(train_test_combined, target_col='sales', windows=[7, 14, 30])
train_test_combined = create_rolling_features(train_test_combined, target_col='n_transactions', windows=[7, 14, 30])
train_test_combined = create_yoy_features(train_test_combined, target_col='sales')
train_test_combined = create_yoy_features(train_test_combined, target_col='n_transactions')

# Test 부분만 추출
test_lag_features = train_test_combined[train_test_combined.index >= len(train)].copy()
test_lag_features = test_lag_features.drop(columns=['sales', 'n_transactions'], errors='ignore')

# Test에 특성 추가
for col in test_lag_features.columns:
    if col not in test.columns and col != 'date' and col != 'store_nbr':
        test[col] = test_lag_features[col].values

print("✓ Test 데이터 시계열 특성 추가 완료")

# 결측치 처리
lag_cols = [col for col in train.columns if 'lag' in col or 'roll' in col or 'yoy' in col]
print(f"\n추가된 총 {len(lag_cols)}개 특성의 결측치 처리 중...")

for col in lag_cols:
    if col in train.columns:
        fill_value = train.groupby('store_nbr')[col].transform('mean')
        train[col].fillna(fill_value, inplace=True)
        train[col].fillna(train[col].mean(), inplace=True)
    
    if col in test.columns:
        fill_value = test.groupby('store_nbr')[col].transform('mean')
        test[col].fillna(fill_value, inplace=True)
        test[col].fillna(test[col].mean(), inplace=True)

print("✓ 결측치 처리 완료")

Test 데이터에 시계열 특성 추가 중...
✓ Test 데이터 시계열 특성 추가 완료

추가된 총 34개 특성의 결측치 처리 중...
✓ 결측치 처리 완료


## 13. Categorical 특성 인코딩

### 📚 설명

**Test 데이터에 시계열 특성을 추가**합니다. 이것이 중요한 이유:

**문제:** Test 데이터의 lag를 계산하려면 Train의 마지막 데이터가 필요
- `test_lag_1` = test의 어제 판매량인데, 그 어제는 Train의 마지막 날일 수 있음
- Test만으로는 계산 불가능!

**해결법:**
1. Train + Test를 **시간순으로 연결**
2. 이 병합 데이터에서 lag/rolling/yoy 특성 계산
3. Test 부분만 추출하여 원래 Test 데이터에 병합

**결측치 처리:**
- 처음 몇 행의 lag값은 `groupby('store_nbr').transform('mean')` 사용
- 각 매장별 평균으로 결측치 채우기 (전체 평균보다 정확함)

**결과:** Test도 Train과 동일한 특성 개수를 가짐

In [13]:
# 카테고리 특성 확인
categorical_cols = ['type', 'family']

print("카테고리 값 개수:")
for col in categorical_cols:
    print(f"{col}: {train[col].nunique()}")

# One-hot encoding
train = pd.get_dummies(train, columns=categorical_cols, drop_first=False)
test = pd.get_dummies(test, columns=categorical_cols, drop_first=False)

# Train과 Test의 컬럼 맞추기
train_cols = set(train.columns)
test_cols = set(test.columns)

for col in test_cols - train_cols:
    test.drop(col, axis=1, inplace=True)

for col in train_cols - test_cols:
    test[col] = 0

test = test[train.columns.drop('sales', errors='ignore')]

print(f"\nTrain shape: {train.shape}")
print(f"Test shape: {test.shape}")

카테고리 값 개수:
type: 5
family: 33

Train shape: (3008016, 97)
Test shape: (28512, 96)


## 14. 결측치 최종 확인

### 📚 설명

**범주형(Categorical) 특성을 인코딩**합니다. 머신러닝 모델은 숫자만 이해하기 때문:

**범주형 특성:**
- `type`: 매장 유형 (A, B, C, D) → 4개 값
- `family`: 상품 분류 (33개 값: Beverages, Dairy, Frozen Food 등)

**One-hot encoding:**
```python
type_A=1, type_B=0, type_C=0, type_D=0  # type='A'인 경우
family_Beverages=1, family_Dairy=0, ...  # family='Beverages'인 경우
```

**Train과 Test 컬럼 맞추기:**
- Test에만 있는 카테고리 제거 (Train과 같은 범주만 유지)
- Train에만 있는 카테고리는 Test에 0으로 추가 (모두 False)
- 컬럼 순서를 동일하게 정렬

**특성 개수:** 4개(type) + 33개(family) = 37개 추가

In [14]:
print("=== Train 결측치 ===")
missing_train = train.isnull().sum()
print(missing_train[missing_train > 0])

print("\n=== Test 결측치 ===")
missing_test = test.isnull().sum()
print(missing_test[missing_test > 0])

if missing_train.sum() == 0:
    print("\n✓ Train에 결측치 없음")
if missing_test.sum() == 0:
    print("✓ Test에 결측치 없음")

=== Train 결측치 ===
Series([], dtype: int64)

=== Test 결측치 ===
Series([], dtype: int64)

✓ Train에 결측치 없음
✓ Test에 결측치 없음


### 📚 설명

**결측치(Missing values)를 최종 확인**합니다. 모델 학습 전 필수 검사:

**결측치가 남아있으면:**
- XGBoost 등 일부 모델: 자동 처리 (성능 저하)
- 다른 모델들: 에러 발생

**확인 항목:**
1. Train의 각 컬럼별 결측치 개수
2. Test의 각 컬럼별 결측치 개수
3. 결측치가 있으면 어디서 왔는지 추적

**이상적인 상태:** 모든 결측치 = 0
- 앞서 interpolate, fillna로 처리했기 때문에 0이어야 함

## 15. 데이터 통계

In [15]:
print("\n=== Train 데이터 정보 ===")
print(f"Shape: {train.shape}")
print(f"컬럼 수: {len(train.columns)}")
print(f"\n수치형 특성 통계:")
print(train.describe())


=== Train 데이터 정보 ===
Shape: (3008016, 97)
컬럼 수: 97

수치형 특성 통계:
                 id                           date     store_nbr  \
count  3.008016e+06                        3008016  3.008016e+06   
mean   1.501508e+06  2015-04-24 22:50:02.843602688  2.750000e+01   
min    0.000000e+00            2013-01-01 00:00:00  1.000000e+00   
25%    7.520038e+05            2014-02-27 18:00:00  1.400000e+01   
50%    1.502226e+06            2015-04-25 12:00:00  2.750000e+01   
75%    2.248883e+06            2016-06-18 06:00:00  4.100000e+01   
max    3.000887e+06            2017-08-15 00:00:00  5.400000e+01   
std    8.657303e+05                            NaN  1.558579e+01   

              sales   onpromotion          year         month           day  \
count  3.008016e+06  3.008016e+06  3.008016e+06  3.008016e+06  3.008016e+06   
mean   3.582691e+02  2.609620e+00  2.014839e+03  6.209123e+00  1.561789e+01   
min    0.000000e+00  0.000000e+00  2.013000e+03  1.000000e+00  1.000000e+00   
25%    

### 📚 설명

**데이터의 통계 정보를 출력**합니다. 데이터 분포를 이해하는 것이 중요:

**확인 항목:**
- `count`: 비결측치 개수 (대부분 같아야 함)
- `mean`: 평균 판매량 (스케일 감잡기)
- `std`: 표준편차 (판매량 변동성)
- `min/max`: 최소/최대값 (이상치 확인)
- `25%, 50%, 75%`: 사분위수 (분포 모양)

**이상치 여부 확인:**
- min이 0이상? → 판매량이 음수일 수 없음 ✓
- max가 너무 크지는 않은가? → 프로모션이나 큰 매장일 수 있음

**스케일링 필요?** 
- 이후 모델 훈련 시 트리 기반 모델(XGBoost)은 스케일링 불필요
- 신경망 등은 필요할 수 있음

## 16. 전처리된 데이터 저장

In [16]:
# 전처리된 데이터 저장
train.to_csv('train_processed.csv', index=False)
test.to_csv('test_processed.csv', index=False)

print("✓ train_processed.csv 저장 완료")
print("✓ test_processed.csv 저장 완료")
print(f"\n총 {len(train)} 행의 학습 데이터 준비 완료")
print(f"총 {len(test)} 행의 테스트 데이터 준비 완료")

KeyboardInterrupt: 

### 📚 설명

**전처리된 데이터를 CSV로 저장**합니다. 다음 단계(모델 훈련)에서 사용:

**저장하는 이유:**
1. **재사용성**: 모델 훈련할 때마다 이 전처리 과정을 반복할 필요 없음
2. **시간 절약**: 다음 노트북에서 바로 모델 훈련 가능
3. **일관성**: 모든 분석이 동일한 데이터셋 기반

**저장되는 파일:**
- `train_processed.csv`: 학습 데이터 (3,000,888 행 × 약 90열)
- `test_processed.csv`: 테스트 데이터 (28,512 행 × 약 90열)

**컬럼 정보:**
- Train: `sales` (target) 포함
- Test: `sales` 미포함 (예측해야 할 값)
- 둘 다: `date`, `store_nbr` 등 메타데이터 포함

## 17. 특성 목록 확인

### 📚 설명

**최종적으로 생성된 모든 특성을 나열**합니다. 모델 훈련의 입력 특성 확인:

**특성 제외 사항:**
- `sales`: 예측 대상 (입력이 아님)
- `date`: 이미 year/month/day로 분해됨
- `id`: 고유 식별자 (예측 성능과 무관)

**특성 종류:**
1. **날짜 특성**: year, month, day, dayofweek, month_sin/cos 등 (13개)
2. **매장 특성**: store_nbr, type_A/B/C/D, cluster, city 등 (10+개)
3. **경제 특성**: oil_price (1개)
4. **휴일**: is_holiday (1개)
5. **거래량**: n_transactions (1개)
6. **Lag**: sales_lag_1/7/14/30, n_transactions_lag_1/7/14/30 (8개)
7. **Rolling**: roll_mean/std/min/max (24개)
8. **YoY**: yoy_365 (2개)
9. **범주형**: family_Beverages, ... (33개)

**총 특성 수: 약 90~100개**
- 초기 컬럼 18개 → 특성 엔지니어링을 통해 5배 이상 증가
- 이것이 모델 성능의 핵심!

In [ ]:
# Train에서 target 제외한 특성 목록
feature_cols = [col for col in train.columns if col not in ['sales', 'date', 'id']]

print(f"\n총 특성 개수: {len(feature_cols)}")
print("\n특성 목록:")
for i, col in enumerate(feature_cols, 1):
    print(f"{i}. {col}")